# Setup
To install all the dependencies and add the enviornment run it in the terminal:

```bash
pip install -r requirements.txt
```

#Connecting Notebook to the .py Files

In [2]:
import sys
import os
import pandas as pd

In [3]:
sys.path.append(os.getcwd())

from recommender import recommend_anime
from database import init_db, create_user, login_user, add_favorite, get_favorites


## Problem Definition

Finding new anime to watch is difficult due to the massive number of available titles. Users often struggle to discover anime that matches their taste.

This project solves this problem by building an intelligent anime recommendation system that suggests similar anime based on genre and description similarity.


In [ ]:
[CSV Dataset] ---> TF-IDF ---> Cosine Similarity ---> Recommendations
                      |
                      v
                Anime ID
                      |
                Jikan API
                      |
            Live synopsis, studio, year


In [4]:
anime = pd.read_csv("anime_with_id.csv")
anime.head()


,anime_id,Anime,Genre,Description,Studio,Year,Rating
0,0,Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season,"['Drama', 'Fantasy', 'Suspense']",A reunion that was supposed to spell the arriv...,White Fox,"Jul 8, 2020",8.37
1,1,"Kanojo, Okarishimasu","['Comedy', 'Romance']",Kazuya Kinoshita is a 20-year-old college stud...,TMS Entertainment,"Jul 11, 2020",NaN
2,2,The God of High School,"['Action', 'Fantasy', 'Sci-Fi', 'Supernatural']","The ""God of High School"" tournament has begun,...",MAPPA,"Jul 6, 2020",NaN
3,3,Maou Gakuin no Futekigousha: Shijou Saikyou no...,"['Action', 'Fantasy']","In the distant past, a war between humans and ...",SILVER LINK.,"Jul 4, 2020",NaN
4,4,Enen no Shouboutai: Ni no Shou,"['Action', 'Supernatural']",After his confrontation in the Nether with his...,David Production,"Jul 4, 2020",NaN


In [5]:
anime.info()

anime.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2981 entries, 0 to 2980
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   anime_id     2981 non-null   int64  
 1   Anime        2981 non-null   object 
 2   Genre        2981 non-null   object 
 3   Description  2981 non-null   object 
 4   Studio       2895 non-null   object 
 5   Year         2981 non-null   object 
 6   Rating       276 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 163.2+ KB


anime_id          0
Anime             0
Genre             0
Description       0
Studio           86
Year              0
Rating         2705
dtype: int64

## Feature Engineering

We combine the Genre and Description columns into a single text field. This allows TF-IDF to capture the semantic similarity between anime.


# Save / Export
If you re-fit TF-IDF or compute artifacts here, save them with `joblib.dump` to `recommender_artifacts.joblib` so the app can load them on startup.

In [ ]:
# Example: save dummy artifacts (uncomment if you have objects to dump)
# from joblib import dump
# dump({'tfidf': tfidf, 'tfidf_matrix': tfidf_matrix, 'cosine_sim': cosine_sim}, 'recommender_artifacts.joblib')
pass

In [6]:

anime['content'] = anime['Genre'].fillna('') + " " + anime['Description'].fillna('')
anime[['Anime', 'content']].head()

,Anime,content
0,Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season,"['Drama', 'Fantasy', 'Suspense'] A reunion tha..."
1,"Kanojo, Okarishimasu","['Comedy', 'Romance'] Kazuya Kinoshita is a 20..."
2,The God of High School,"['Action', 'Fantasy', 'Sci-Fi', 'Supernatural'..."
3,Maou Gakuin no Futekigousha: Shijou Saikyou no...,"['Action', 'Fantasy'] In the distant past, a w..."
4,Enen no Shouboutai: Ni no Shou,"['Action', 'Supernatural'] After his confronta..."


# Recommendation Model

We use:
- TF-IDF Vectorization
- Cosine Similarity

Anime with similar genre and description vectors are considered similar and recommended.


In [7]:
#Test Recommendation
recommend_anime("Naruto")

,Anime,Genre,Similarity Score
0,Boruto: Naruto Next Generations,"['Action', 'Adventure', 'Fantasy']",0.519
1,Naruto: Shippuuden,"['Action', 'Adventure', 'Fantasy']",0.475
2,Mayoiga,"['Drama', 'Horror', 'Mystery']",0.191
3,Nyamen,Comedy,0.164
4,H2O: Footprints in the Sand,Romance,0.138


# Evaluation

The system is evaluated qualitatively by checking whether recommended anime are similar in genre and theme to the input anime.


In [8]:
recommend_anime("Death Note")


,Anime,Genre,Similarity Score
0,Mobile Suit Gundam SEED,"['Action', 'Drama', 'Romance', 'Sci-Fi']",0.142
1,gdMen,"['Comedy', 'Fantasy']",0.141
2,Soul Eater,"['Action', 'Comedy', 'Fantasy', 'Supernatural']",0.141
3,Ji Jia Shou Shen Zhi Baolie Feiche,"['Action', 'Adventure']",0.141
4,Shadowverse Flame,Fantasy,0.133


In [10]:
#Initialize database and create/login a user
init_db()

create_user("demo", "demo123")
uid = login_user("demo", "demo123")
uid

1

User passwords are securely stored using PBKDF2 hashing instead of plaintext, ensuring data privacy and protection.

In [11]:
#Add and get favorites
add_favorite(uid, 1, "Naruto")
get_favorites(uid)

Error adding favorite: database is locked


[(b'\x1a\x00\x00\x00\x00\x00\x00\x00', 'Black Clover')]

# Ethical Considerations

The dataset may contain popularity bias. Lesser-known anime may be under-recommended. The system is designed only for entertainment and does not collect sensitive personal data.


# Conclusion & Future Scope

This system successfully recommends anime based on content similarity.

Future improvements:
- User-based collaborative filtering
- Deep learning embeddings
- Mobile app (APK)

# Live Hosted App
https://animexrecommender-nexus.streamlit.app/